# 03_funcpp02

功能像预处理的最核心部分。这一部分我们完成一下步骤
* Despike
* Slice-timing correction (时间层矫正)
* Distortion correction (图像畸变矫正)
* Motion correction (头动矫正)
* align anat 2 epi (结构像到功能像的配准) 
* 把所有epi划到个体结构像空间
* 把所有epi的volume数据划到surface data
* 把所有epi的划到MNI标准空间
* Scale (标准化数据尺度，抓换成percent signal change)

## Step 1. Despike

In [ ]:
# ================================ despike =================================
for run in runstr:
    ! 3dDespike -NEW -nomask -prefix pb01.{subj}.r{run}.despike pb00.{subj}.r{run}.tcat+orig
! rm pb00*

## Step 2. Slice-timing correction

In [ ]:
# ================================= tshift =================================
for run in runstr:
    ! 3dTshift -tzero 0 -quintic -prefix pb02.{subj}.r{run}.tshift \
         -verbose -tpattern @{'../rawdata/SliceTiming.txt'} pb01.{subj}.r{run}.despike+orig

# 删掉上一步despike的data节省空间
! rm pb01*

## Step 3. Distortion correction

因为磁场不均匀的关系，图像会出现畸变。现在来做矫正。

In [ ]:
# 首先找到
for fwd,rev in zip(fwdfiles, revfiles):
        # create median datasets from forward and reverse time series
        ! 3dTstat -median -prefix rm.blip.med.fwd.r{fwd} pb02.{subj}.r{fwd}.tshift+orig
        ! 3dTstat -median -prefix rm.blip.med.rev.r{rev} pb02.{subj}.r{rev}.tshift+orig
        # automask the median datasets 
        ! 3dAutomask -apply_prefix rm.blip.med.masked.fwd.r{fwd} rm.blip.med.fwd.r{fwd}+orig
        ! 3dAutomask -apply_prefix rm.blip.med.masked.rev.r{rev} rm.blip.med.rev.r{rev}+orig
        # compute the midpoint warp between the median datasets
        ! 3dQwarp -plusminus -pmNAMES Rev.r{rev} Fwd.r{fwd}                           \
                -pblur 0.05 0.05 -blur -1 -1                          \
                -noweight -minpatch 9                                 \
                -noXdis -noZdis                                       \
                -source rm.blip.med.masked.rev.r{rev}+orig                   \
                -base   rm.blip.med.masked.fwd.r{fwd}+orig                   \
                -prefix blip_warp
        
        ! 3dNwarpApply -quintic -nwarp blip_warp_Fwd.r{fwd}_WARP+orig        \
                -source rm.blip.med.masked.fwd.r{fwd}+orig               \
                -prefix rm.blip.med.masked.fwd.post.r{fwd}

        ! 3dNwarpApply -quintic -nwarp blip_warp_Rev.r{rev}_WARP+orig        \
                -source rm.blip.med.masked.rev.r{rev}+orig               \
                -prefix rm.blip.med.masked.rev.post.r{rev}
        
        # 删掉多余文件
        ! rm blip_warp_Fwd.r{fwd}+orig* blip_warp_Rev.r{rev}+orig* 

        # 修改个名字以便以后的操作
        ! 3dcopy blip_warp_Fwd.r{fwd}_WARP+orig blip_warp.r{fwd}_WARP+orig
        ! 3dcopy blip_warp_Rev.r{rev}_WARP+orig blip_warp.r{rev}_WARP+orig

        ! rm blip_warp_Fwd.r{fwd}+orig* blip_warp_Rev.r{rev}+orig* blip_warp_Fwd.r{fwd}_WARP+orig* blip_warp_Rev.r{rev}_WARP+orig* 

# for QC check，我们生成一个校正前和矫正后的平均EPI数据来作为distortion correction的效果证明
! 3dMean -prefix blip_pre_fwd.epi_mean.nii.gz rm.blip.med.masked.fwd.r*
! 3dMean -prefix blip_pre_rev.epi_mean.nii.gz rm.blip.med.masked.rev.r*
! 3dMean -prefix blip_post_fwd.epi_mean.nii.gz rm.blip.med.masked.fwd.post.r*
! 3dMean -prefix blip_post_rev.epi_mean.nii.gz rm.blip.med.masked.rev.post.r*

# remove redundent file
! rm rm*

这一步完成之后，可以手动打开
* blip_pre_fwd.epi_mean.nii.gz
* blip_pre_rev.epi_mean.nii.gz
* blip_post_fwd.epi_mean.nii.gz
* blip_post_rev.epi_mean.nii.gz

这四个文件也记录了对侧矫正的效果，只要有矫正效果，没有什么大问题即可。

然后把所有的run都做correction

In [ ]:
for run in runstr:
    ! 3dNwarpApply -quintic -nwarp blip_warp.r{run}_WARP+orig      \
            -source pb02.{subj}.r{run}.tshift+orig         \
            -prefix pb03.{subj}.r{run}.blip

## Step 4. Motion correction 

首先做motion correction。但是我们并不用其结果，主要是得到mc的线性转换矩阵

In [ ]:
# ================================= volreg =================================
base = 'vr_base_min_outlier+orig' #在前面生成

# align each dset to base volume, then we separate whether we want to further align epi to anat
for run in runstr:
    # register each volume to the base image
    ! 3dvolreg -verbose -zpad 1 -base {base} \
             -1Dfile dfile.r{run}.1D \
             -cubic \
             -1Dmatrix_save mat.r{run}.vr.aff12.1D \
             -prefix rm.epi.nomask.r{run} pb03.{subj}.r{run}.blip+orig

    # create an all-1 dataset to mask the extents of the warp
    ! 3dcalc -overwrite -a pb03.{subj}.r{run}.blip+orig -expr 1 -prefix rm.epi.all1

    # warp the all-1 dataset for extents masking
    ! 3dAllineate -base {base} \
                -input rm.epi.all1+orig \
                -1Dmatrix_apply mat.r{run}.vr.aff12.1D \
                -final NN -quiet \
                -prefix rm.epi.1.r{run}

    # make an extents intersection mask of this run across time domain
    # this makes a mask that all volumes in this run have valid numbers
    ! 3dTstat -min -prefix rm.epi.min.r{run} rm.epi.1.r{run}+orig

计算更多的头动数值，以后可以用来GLM中的regressor

In [ ]:
# --------------- deal with motion parameter -------------------
# we take dmeaned motion (6), demean motion derivative (6) and their squares (12)
# 合并 motion params
! cat dfile.r*.1D > dfile_rall.1D
# 计算去均值的motion parameters (for use in regression)
! 1d_tool.py -infile dfile_rall.1D -set_nruns {nRuns} -demean -write motion_demean.1D
# 计算一阶导数 (just to have)
! 1d_tool.py -infile dfile_rall.1D -set_nruns {nRuns} -derivative -demean -write motion_deriv.1D
# calculate the square of demean and derivative of motion parameters (just to have)
# for resting preproc, we usually have 24 motion regressor (6motion+6deriv+12 their square)
np.savetxt('motion_demeansq.1D', np.loadtxt('motion_demean.1D')**2)
np.savetxt('motion_derivsq.1D', np.loadtxt('motion_deriv.1D')**2)
# create censor file motion_${subj}_censor.1D, for censoring motion
! 1d_tool.py -infile dfile_rall.1D -set_nruns {nRuns} \
   -show_censor_count -censor_prev_TR \
   -censor_motion {motion_censor} motion_{subj}

# Estimate the motion parameter after motion correction, we can check the
# effects of MC
! mkdir mc
for run in runstr:
    ! 3dvolreg -verbose -zpad 1 -base {base} \
             -1Dfile dfile.r{run}_pos.1D \
             rm.epi.nomask.r{run}+orig
    ! rm volreg+orig*
# make a single file of motion params
! cat dfile.r*_pos.1D > dfile_rall_pos.1D

# make figure pre mc
dfile_pre = np.loadtxt('dfile_rall.1D')
plot(range(dfile_pre.shape[0]), dfile_pre[:,:3], color=['C0','C1','C2'], label=['roll(IS)','pitch(RL)','yaw(AP)'])
plt.legend();plt.xlabel('time points');plt.ylabel('mm');
plt.savefig('rots_pre.pdf');plt.close('all')
plot(range(dfile_pre.shape[0]), dfile_pre[:,3:], color=['C3','C4','C5'], label=['dS','dL','dP'])
plt.legend();plt.xlabel('time points (TR)');plt.ylabel('mm');
plt.savefig('tran_pre.pdf');plt.close('all')
# make figure pos mc
dfile_pos = np.loadtxt('dfile_rall_pos.1D')
plot(range(dfile_pos.shape[0]), dfile_pos[:,:3], color=['C0','C1','C2'], label=['roll(IS)','pitch(RL)','yaw(AP)'])
plt.legend();plt.xlabel('time points');plt.ylabel('mm');
plt.savefig('rots_pos.pdf');plt.close('all')
plot(range(dfile_pos.shape[0]), dfile_pos[:,3:], color=['C3','C4','C5'], label=['dS','dL','dP'])
plt.legend();plt.xlabel('time points (TR)');plt.ylabel('mm');
plt.savefig('tran_pos.pdf');plt.close('all')
# move file to directory
! mv rots*.pdf tran*.pdf dfile.r*_pos.1D dfile_r*_pos.1D mc/

# note TRs that were not censored, note ktrs here is a str
ktrs = unix_wrapper(f'1d_tool.py -infile motion_{subj}_censor.1D \
                       -show_trs_uncensored encoded', wantreturn=True, verbose=0)

# ----------------------------------------
# create the extents mask: mask_epi_extents+orig and apply the task
# (this is a mask of voxels that have valid data at every TR,
# there might be some pixel out of extents during mc)
! 3dMean -datum short -prefix rm.epi.mean rm.epi.min.r*.HEAD
! 3dcalc -a rm.epi.mean+orig -expr "step(a-0.999)" -prefix mask_epi_extents
# and apply the extents mask to the EPI data
# (delete any time series with missing data)
for run in runstr:
    ! 3dcalc -a rm.epi.nomask.r{run}+orig -b mask_epi_extents+orig \
           -expr "a*b" -prefix pb04.{subj}.r{run}.volreg
! rm -f rm.*  # rm.epi.nomask are big files, remove them
! rm -f {base}* 

# calculate a mean epi volume for next step anat epi registration
! 3dMean -prefix rm.epi_mean.nii.gz pb04.{subj}.r*.volreg+orig.HEAD
! 3dTstat -prefix epi_mean.nii.gz rm.epi_mean.nii.gz
! rm rm* mask_epi_extents+orig*

# 删掉上一步distortion correction的数据，节省空间
! rm pb03*

## Step 5. anat2epi

这一步非常重要，是前面处理好的功能像和结构像进行配准，配准的结果一定要手动检查

In [ ]:
# 首先把用来配准的epi像make a copy
! 3dcopy epi_mean.nii.gz epi_mean_tmp.nii.gz

In [ ]:

# 关键步骤。我们是把结构像配准到功能像，但是我们得到的转换矩阵是从相反的，功能像到结构像，这个矩阵可以后面用来转化epi数据
! align_epi_anat.py -anat2epi \
    -anat {t1ss} -anat_has_skull no \
    -suffix _al_junk \
    -epi epi_mean_tmp.nii.gz -epi_base 0 \
    -epi_strip 3dAutomask -giant_move \
    -volreg off -tshift off

# 生成nifti文件，方便我们在fsleyes上面查看配准效果
! 3dcopy {t1ss.pstem}_al_junk+orig {t1ss.pstem}_al_junk.nii.gz


完成上面一步之后，要打开afni仔细检查配准的结果，其中underlay选择CN003_SurfVol_al_junk+orig, overlay选择epi_mean_tmp+orig。然后检查两者是否一致。如果不匹配，则需要重新进行上面的过程，在开始之前，需要先删除上面产生的文件。可以做
! rm {subj}_SurfVol_al* {subj}_SurfVol_ns*
确认配准没有问题之后，进行下一步操作

In [ ]:

# note that we align anat 2 epi, however, this xfm is from epi space to anat space
# note that this xfm is compatible with LPS+ space, which is the default space in AFNI
! mv {t1ss.pstem}_al_junk_mat.aff12.1D mat.anat2epi.aff12.1D

#######去头皮后的结构像就是t1ss({subj}_SurfVol_SS.nii), 不需要再生成
# create an skull-striped anat_final dataset, aligned with stats
#! 3dcopy {t1ss.pstem}_ns+orig anat_final.{subj}.nii.gz
#! rm {t1ss.pstem}_ns+orig*
! 3dcopy {t1ss} anat_final.{subj}.nii.gz
########

# rewrite the name of aligned anat
! 3dcopy {t1ss.pstem}_al_junk+orig anat2epi.{subj}.nii.gz
! rm {t1ss.pstem}_al_junk+orig*

# Invert xfm
! cat_matvec -ONELINE mat.anat2epi.aff12.1D -I > mat.epi2anat.aff12.1D

# warp the volreg base EPI dataset back to anat to make a final version
! 3dAllineate -base anat_final.{subj}.nii.gz \
            -input epi_mean.nii.gz \
            -1Dmatrix_apply mat.epi2anat.aff12.1D \
            -prefix epi2anat.{subj}.nii.gz

# Record final registration costs
! 3dAllineate -base epi2anat.{subj}.nii.gz -allcostX -input anat_final.{subj}.nii.gz > out.allcostX.txt

# Take the snapshots to show the quality of alignment
! @snapshot_volreg epi2anat.{subj}.nii.gz anat_final.{subj}.nii.gz
! @snapshot_volreg anat_final.{subj}.nii.gz epi2anat.{subj}.nii.gz
! @snapshot_volreg anat2epi.{subj}.nii.gz epi_mean.nii.gz
! @snapshot_volreg epi_mean.nii.gz anat2epi.{subj}.nii.gz


## Step 6. epi to individual anat

现在我们有了epi到anat的配准，然后我们就可以把所有的epi的文件划到t1的空间。注意，这一步并不是从motion correction之后的文件，也就是pb03.CN003.r0X.volreg+orig。我们要从早distrotion correction之前的文件，把从DC-MC-epi2anat这三个转换一次性的做完

In [ ]:
# ================================= epi to individual anat =================================
# resolution=2.5  # 改到最开始的参数设置
# ======== Transform epi to match anat, add by RZ ===============
# In this step we need to concatenate Distortion(opt)+motion+affine transformations
# align each dset to base volume, then we separate whether we want to further align epi to anat
for run in runstr:
    # concatenate MC+aff transforms
    ! cat_matvec -ONELINE mat.anat2epi.aff12.1D -I mat.r{run}.vr.aff12.1D > mat.r{run}.warp.aff12.1D

    # we apply distortion correction + motion correction + anat2epi transformation
    cmd = f'3dNwarpApply -quintic -nwarp "mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master {t1ss} -dxyz {resolution} \
            -source pb02.{subj}.r{run}.tshift+orig         \
            -prefix rm.epi.nomask.r{run}'
    unix_wrapper(cmd)

    # create an all-1 dataset to mask the extents of the warp
    ! 3dcalc -overwrite -a pb02.{subj}.r{run}.tshift+orig -expr 1 -prefix rm.epi.all1

    # we apply distortion correction + motion correction + anat2epi transformation
    cmd = f'3dNwarpApply -quintic -nwarp "mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master {t1ss} -dxyz {resolution} \
            -ainterp NN -quiet \
            -source rm.epi.all1+orig \
            -prefix rm.epi.1.r{run}'
    unix_wrapper(cmd)

    # make an extents intersection mask of this run across time domain
    # this makes a mask that all volumes in this run have valid numbers
    ! 3dTstat -min -prefix rm.epi.min.r{run} rm.epi.1.r{run}+orig
    # below file is big, should be removed
    ! rm rm.epi.1.r{run}+orig*


做了变化之后，也要处理一下mask的问题

In [ ]:

# ----------------------------------------
# create the extents mask: mask_epi_extents+orig
# (this is a mask of voxels that have valid data at every TR,
# there might be some pixel out of extents during mc)
! 3dMean -datum short -prefix rm.epi.mean rm.epi.min.r*.HEAD
# and apply the extents mask to the EPI data
! 3dcalc -a rm.epi.mean+orig -expr "step(a-0.999)" -prefix mask_epi_extents

# 去掉mask之外的voxel的数据
for run in runstr:
    ! 3dcalc -a rm.epi.nomask.r{run}+orig -b mask_epi_extents+orig \
           -expr "a*b" -prefix pb05.{subj}.r{run}.al2anat
    # save to nifti format seems to reduce file size

# rm.epi.nomask are big files, remove them
! rm rm.*


## Step 7. epi volume 2 surface

上一步我们已经把epi数据划到了和个体的结构像一个空间。很多时候我们需要做基于surface的数据分析。我们进一步把三维的epi数据插值划到surface上

In [ ]:

# ================================= epi volume to surface =================================
surface_dir = FREESURFER_HOME+f'/subjects/{subj}/SUMA'
# map volume data to the surface of each hemisphere
for hemi in ['lh', 'rh']:
    for  run in runstr:
        ! 3dVol2Surf -spec {surface_dir}/std.141.{subj}_{hemi}.spec   \
                   -sv {subj}_SurfVol_2std.nii           \
                   -surf_A smoothwm                            \
                   -surf_B pial                                \
                   -f_index nodes                              \
                   -f_steps 10                                 \
                   -map_func ave                               \
                   -oob_value 0                                \
                   -grid_parent pb05.{subj}.r{run}.al2anat+orig   \
                   -out_niml pb05.{subj}.{hemi}.r{run}.surf.niml.dset



## Step 8. epi to MNI anat

In [ ]:

# ================================= epi to MNI anat =================================
# ======== Transform epi to match anat, add by RZ ===============
# In this step we need to concatenate Distortion(opt)+motion+affine transformations
# align each dset to base volume, then we separate whether we want to further align epi to anat
for run in runstr:

    # we apply distortion correction + motion correction + anat2epi + nonlinear warp to MNI transformation
    # 注意我们这里是相反的顺序来concatenate的
    cmd = f'3dNwarpApply -quintic -nwarp "anatQQ.{subj}_WARP.nii anatQQ.{subj}.aff12.1D mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master MNI152_2009_template_SSW.nii.gz \
            -dxyz {resolution} \
            -source pb02.{subj}.r{run}.tshift+orig \
            -prefix rm.epi.nomask.r{run}'
    unix_wrapper(cmd)

    # create an all-1 dataset to mask the extents of the warp
    ! 3dcalc -overwrite -a pb02.{subj}.r{run}.tshift+orig -expr 1 -prefix rm.epi.all1

    # we apply distortion correction + motion correction + anat2epi transformation
    cmd = f'3dNwarpApply -quintic -nwarp "anatQQ.{subj}_WARP.nii anatQQ.{subj}.aff12.1D mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master MNI152_2009_template_SSW.nii.gz \
            -dxyz {resolution} \
            -source rm.epi.all1+orig         \
            -ainterp NN -quiet \
            -prefix rm.epi.1.r{run}'
    unix_wrapper(cmd)

    # make an extents intersection mask of this run across time domain
    # this makes a mask that all volumes in this run have valid numbers
    ! 3dTstat -min -prefix rm.epi.min.r{run} rm.epi.1.r{run}+tlrc
    # below file is big, should be removed
    ! rm rm.epi.1.r{run}+tlrc*      


做了变化之后，也要处理一下mask的问题

In [ ]:
# ----------------------------------------
# create the extents mask: mask_epi_extents+orig
# (this is a mask of voxels that have valid data at every TR,
# there might be some pixel out of extents during mc)
! 3dMean -datum short -prefix rm.epi.mean rm.epi.min.r*.HEAD
! 3dcalc -a rm.epi.mean+tlrc -expr "step(a-0.999)" -prefix mask_epi_2mni_extents

# 去掉mask之外的voxel的数据
for run in runstr:
    ! 3dcalc -a rm.epi.nomask.r{run}+tlrc -b mask_epi_2mni_extents+tlrc \
           -expr "a*b" -prefix pb06.{subj}.r{run}.al2mni
    # save to nifti format seems to reduce file size

# rm.epi.nomask are big files, remove them
! rm rm.*


In [ ]:
# 补充步骤，用于进行smooth操作，  2022.7.3添加
# 目的是对上一步step8所生成的pb06进行smooth   生成pb06s
# 再对上上一步step7所生成的pb05***surf.niml文件进行smooth  生成pb05s*rh  和 pb05s*lh
# 再对上上一步step7所生成的pb05.subj.r0*.al2anat+orig文件进行smooth  生成pb05s*.subj.r0*.blur+al2anat+orig

for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb06s.{subj}.r{run}.blur+al2mni+tlrc \
        pb06.{subj}.r{run}.al2mni+tlrc

for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb05s.{subj}.lh.r{run}.blur.surf.niml.dset \
        pb05.{subj}.lh.r{run}.surf.niml.dset

for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb05s.{subj}.rh.r{run}.blur.surf.niml.dset \
        pb05.{subj}.rh.r{run}.surf.niml.dset

#以下代码可以不用跑
for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb05s.{subj}.r{run}.blur+al2anat+orig \
        pb05.{subj}.r{run}.al2anat+orig



'''  
#afni官网上的对于surface数据进行smooth的代码

foreach hemi ( lh rh )
    foreach run ( $runs )
        # to save time, estimate blur parameters only once
        if ( ! -f surf.smooth.params.1D ) then
            SurfSmooth -spec $surface_dir/std.60.FT_${hemi}.spec         \
                       -surf_A smoothwm                                  \
                       -input pb03.$subj.$hemi.r$run.surf.niml.dset      \
                       -met HEAT_07                                      \
                       -target_fwhm 6.0                                  \
                       -blurmaster pb03.$subj.$hemi.r$run.surf.niml.dset \
                       -detrend_master                                   \
                       -output pb04.$subj.$hemi.r$run.blur.niml.dset     \
                       | tee surf.smooth.params.1D
        else
            set params = `1dcat surf.smooth.params.1D`
            SurfSmooth -spec $surface_dir/std.60.FT_${hemi}.spec         \
                       -surf_A smoothwm                                  \
                       -input pb03.$subj.$hemi.r$run.surf.niml.dset      \
                       -met HEAT_07                                      \
                       -Niter $params[1]                                 \
                       -sigma $params[2]                                 \
                       -output pb04.$subj.$hemi.r$run.blur.niml.dset
        endif
    end
end
'''

## Step 9. 创造一个所有epi和mni模板共有的mask

In [ ]:
# ====================== Mask =================================
# Create 'full_mask' dataset (union mask)
for run in runstr:
    ! 3dAutomask -prefix rm.mask_r{run}.nii.gz pb06.{subj}.r{run}.al2mni+tlrc

# 创造一个所有功能像的run合起来的mask
! 3dmask_tool -inputs rm.mask_r*.nii.gz -union -prefix full_mask.{subj}.nii.gz
! rm rm.mask*

# ---- create subject anatomy mask, mask_anat.$subj+orig ----
#      (resampled from aligned anat)
# 把功能像resample到上面mask的分辨率
! 3dresample -master full_mask.{subj}.nii.gz -input \
           MNI152_2009_template_SSW.nii.gz -prefix rm.resam.anat.nii.gz
# convert to binary anat mask; fill gaps and holes
! 3dmask_tool -dilate_input 5 -5 -fill_holes -input rm.resam.anat.nii.gz \
            -prefix mask_anat.{subj}.nii.gz

# 结合功能像和结构像的mask
# compute tighter EPI mask by intersecting with anat mask
! 3dmask_tool -input full_mask.{subj}.nii.gz mask_anat.{subj}.nii.gz \
            -inter -prefix mask_epi_anat.{subj}.nii.gz
# note Dice coefficient of masks, as well
! 3ddot -dodice full_mask.{subj}.nii.gz mask_anat.{subj}.nii.gz > out.mask_ae_dice.txt

# 删掉多余文件
! rm rm*

## Step 10. scale

原始EPI图像的数据可能很大，从几百到4000不等。但是我们一般不考虑这个绝对强度，考虑的是信号增长的比例。所以把数据scale一下, 让所有的voxel的时间序列均值为100

In [ ]:
# 注意！！！这个cell的代码是对pb06以及pb05 进行scale重置 也就是未经smooth的数据
# scale each voxel time series to have a mean of 100
# (be sure no negatives creep in)
# (subject to a range of [0,200])
# scale volume data

for run in runstr:
    ! 3dTstat -prefix rm.mean_r{run} pb06.{subj}.r{run}.al2mni+tlrc
    ! 3dcalc -a pb06.{subj}.r{run}.al2mni+tlrc -b rm.mean_r{run}+tlrc \
           -c mask_epi_2mni_extents+tlrc \
           -expr "c * min(200, a/b*100)*step(a)*step(b)" \
           -prefix pb07.{subj}.r{run}.scale

# combine all datasets into one
! 3dTcat -prefix all_runs.{subj}.nii.gz pb07.{subj}.r*.scale+tlrc*
# remove redundant file
! rm rm.mean*

# 再scale surface data
for hemi in ['lh', 'rh']:
    for run in runstr:
       ! 3dTstat -prefix rm.{hemi}.mean_r{run}.niml.dset    \
            pb05.{subj}.{hemi}.r{run}.surf.niml.dset
       ! 3dcalc -a pb05.{subj}.{hemi}.r{run}.surf.niml.dset  \
               -b rm.{hemi}.mean_r{run}.niml.dset          \
               -expr 'min(200, a/b*100)*step(a)*step(b)' \
               -prefix pb07.{subj}.{hemi}.r{run}.scale.niml.dset

# remove redundant file
! rm rm.*mean* 


In [ ]:
# 注意！！！这个cell的代码是对pb06s 以及pb05s 进行scale重置 也就是经过了smooth的数据
# 生成pb07s*.scale  
# 这个文件被整合成all_runs_with_smooth
# 生成pb07s*.scale.niml.dset

# scale volume data
for run in runstr:
    ! 3dTstat -prefix rm.mean_r{run} pb06s.{subj}.r{run}.blur+al2mni+tlrc
    ! 3dcalc -a pb06s.{subj}.r{run}.blur+al2mni+tlrc -b rm.mean_r{run}+tlrc \
           -c mask_epi_2mni_extents+tlrc \
           -expr "c * min(200, a/b*100)*step(a)*step(b)" \
           -prefix pb07s.{subj}.r{run}.scale

# combine all datasets into one
! 3dTcat -prefix all_runs_with_smooth.{subj}.nii.gz pb07s.{subj}.r*.scale+tlrc*
# remove redundant file
! rm rm.mean*

# 再scale surface data
for hemi in ['lh', 'rh']:
    for run in runstr:
       ! 3dTstat -prefix rm.{hemi}.mean_r{run}.niml.dset    \
            pb05s.{subj}.{hemi}.r{run}.blur.surf.niml.dset
       ! 3dcalc -a pb05s.{subj}.{hemi}.r{run}.blur.surf.niml.dset  \
               -b rm.{hemi}.mean_r{run}.niml.dset          \
               -expr 'min(200, a/b*100)*step(a)*step(b)' \
               -prefix pb07s.{subj}.{hemi}.r{run}.scale.niml.dset
# remove redundant file
! rm rm.*mean* 

到这一步，基本上最主要的预处理就完结了。下一步可以进一步针对surface数据跑GLM，因为afni的surface都是在freesurfer的模板上对其且标准化的，所以surface数据可以直接跑group analysis。也可以在已经划到MNI space上的volume数据跑GLM。以后在volume上跑GLM的结果可以直接用MNI152的T1像来可视化